In [1]:
from itertools import chain
import pandas as pd
import pickle

import nltk
import sklearn                                                              # scikit-learn learn
import scipy.stats
from sklearn.metrics import make_scorer
            
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import RandomizedSearchCV 

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [2]:
with open("C:\\Users\\ADMIN\\Desktop\\My Code\\Try NER\\NER Dataset\\final conll 2002 dataset\\espanish\\esp_train_pos_set.pkl", 'rb') as fp:        # unpickling       notice    r   for read
    train_sents = pickle.load(fp)
    
with open("C:\\Users\\ADMIN\\Desktop\\My Code\\Try NER\\NER Dataset\\final conll 2002 dataset\\espanish\\esp_testa_pos_set.pkl", 'rb') as fp:        # unpickling       notice    r   for read
    testa_sents = pickle.load(fp)
    
with open("C:\\Users\\ADMIN\\Desktop\\My Code\\Try NER\\NER Dataset\\final conll 2002 dataset\\espanish\\esp_testb_pos_set.pkl", 'rb') as fp:        # unpickling       notice    r   for read
    testb_sents = pickle.load(fp)

In [3]:
print(train_sents[0])

print("\n", testa_sents[0] )

print("\n", testb_sents[0] )


[('Melbourne', 'NP', 'B-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'B-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')]

 [('Sao', 'NC', 'B-LOC'), ('Paulo', 'VMI', 'I-LOC'), ('(', 'Fpa', 'O'), ('Brasil', 'NC', 'B-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('23', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFECOM', 'NP', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')]

 [('La', 'DA', 'B-LOC'), ('Coruña', 'NC', 'I-LOC'), (',', 'Fc', 'O'), ('23', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFECOM', 'NP', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')]


In [4]:
def get_word_type(word):
    if word.isdigit(): 
        return "Digit"
    elif word.isalpha():
        return "Alpha"
    elif word.isalnum():
        return "Alnum"
    else:
        return "None"

def get_word_case(word):
    if word.istitle():
        return "Title"
    elif word.isupper():
        return "Upper"
    elif word.islower(): 
        return "Lower"
    else:
        return "None"
    
def get_pattern(token):
    r = ''
    for c in token:
        if c.isupper():
            r += 'U'
        elif c.islower():
            r += 'L'
        elif c.isdigit():
            r += 'D'
        elif c in ('.', ','):
            r += '.'
        elif c in (';', ':', '?', '!'):
            r += ';'
        elif c in ('+', '*', '/', '=', '|', '_', '#'):
            r += '#'
        elif c in ('-'):
            r += '-'
        elif c in ('(', '{', '[', '<'):
            r += '('
        elif c in (')', '}', ']', '>'):
            r += ')'
        else:
            r += c
    return r    

def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word.len()': len(word),
        'word[-4:]': word[-4:],
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word[:2]': word[:2],
        'word[:3]': word[:3],
        'word[:4]': word[:4],
        'word.type': get_word_type(word),
        'word.case()': get_word_case(word),
        'word.pattern()': get_pattern(word),
        'postag': postag,
        'postag[:2]': postag[:2]        
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.len()': len(word1),
            '-1:word.type': get_word_type(word1),
            '-1:word.case()': get_word_case(word1),
            '-1:word.pattern()': get_pattern(word1),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2]
        })
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.len()': len(word1),
            '+1:word.type': get_word_type(word1),
            '+1:word.case()': get_word_case(word1),
            '+1:word.pattern()': get_pattern(word1),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2]
        })
    else:
        features['EOS'] = True
        
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [5]:
sent2features(train_sents[0])[2]

{'bias': 1.0,
 'word.lower()': 'australia',
 'word.len()': 9,
 'word[-4:]': 'alia',
 'word[-3:]': 'lia',
 'word[-2:]': 'ia',
 'word[:2]': 'Au',
 'word[:3]': 'Aus',
 'word[:4]': 'Aust',
 'word.type': 'Alpha',
 'word.case()': 'Title',
 'word.pattern()': 'ULLLLLLLL',
 'postag': 'NP',
 'postag[:2]': 'NP',
 '-1:word.lower()': '(',
 '-1:word.len()': 1,
 '-1:word.type': 'None',
 '-1:word.case()': 'None',
 '-1:word.pattern()': '(',
 '-1:postag': 'Fpa',
 '-1:postag[:2]': 'Fp',
 '+1:word.lower()': ')',
 '+1:word.len()': 1,
 '+1:word.type': 'None',
 '+1:word.case()': 'None',
 '+1:word.pattern()': ')',
 '+1:postag': 'Fpt',
 '+1:postag[:2]': 'Fp'}

In [6]:
#%%time
X_train = [sent2features(s) for s in train_sents]                        # 8323 sentence
y_train = [sent2labels(s) for s in train_sents]

X_testa = [sent2features(s) for s in testa_sents]                        # 1915 sentence
y_testa = [sent2labels(s) for s in testa_sents]

X_testb = [sent2features(s) for s in testb_sents]                        # 1517 sentence
y_testb = [sent2labels(s) for s in testb_sents]

print (X_train[0])
print("\n\n")
print (X_train[1])
print("\n\n")
print (X_train[2])

[{'bias': 1.0, 'word.lower()': 'melbourne', 'word.len()': 9, 'word[-4:]': 'urne', 'word[-3:]': 'rne', 'word[-2:]': 'ne', 'word[:2]': 'Me', 'word[:3]': 'Mel', 'word[:4]': 'Melb', 'word.type': 'Alpha', 'word.case()': 'Title', 'word.pattern()': 'ULLLLLLLL', 'postag': 'NP', 'postag[:2]': 'NP', 'BOS': True, '+1:word.lower()': '(', '+1:word.len()': 1, '+1:word.type': 'None', '+1:word.case()': 'None', '+1:word.pattern()': '(', '+1:postag': 'Fpa', '+1:postag[:2]': 'Fp'}, {'bias': 1.0, 'word.lower()': '(', 'word.len()': 1, 'word[-4:]': '(', 'word[-3:]': '(', 'word[-2:]': '(', 'word[:2]': '(', 'word[:3]': '(', 'word[:4]': '(', 'word.type': 'None', 'word.case()': 'None', 'word.pattern()': '(', 'postag': 'Fpa', 'postag[:2]': 'Fp', '-1:word.lower()': 'melbourne', '-1:word.len()': 9, '-1:word.type': 'Alpha', '-1:word.case()': 'Title', '-1:word.pattern()': 'ULLLLLLLL', '-1:postag': 'NP', '-1:postag[:2]': 'NP', '+1:word.lower()': 'australia', '+1:word.len()': 9, '+1:word.type': 'Alpha', '+1:word.case(

In [7]:
n_sent_train = len(X_train)
print(n_sent_train)

8323


# Selecting 1 % sentences as seed for training


In [8]:
import math
n_seed = math.floor(int(n_sent_train)/100)
print(n_seed)


83


In [9]:
import numpy as np
# Set our RNG seed for reproducibility.
RANDOM_STATE_SEED = 123
np.random.seed(RANDOM_STATE_SEED)

In [10]:
# Isolate our examples for our labeled dataset.
n_labeled_examples = n_sent_train

training_indices = np.random.choice(n_labeled_examples, n_seed , replace=False)

training_indices = list(set(training_indices))
print(len(training_indices))                                            # print 1
initial_X_train = [X_train[x] for x in training_indices]      
initial_Y_train = [y_train[x] for x in training_indices]

# Isolate the non-training examples we'll be querying.
X_pool = np.delete(X_train, training_indices).tolist()
y_pool = np.delete(y_train, training_indices).tolist()


print(len(X_pool))                                                       # print 2
print(len(y_pool))                                                       # print 3



83
8240
8240


In [11]:
#%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True,                             # all_possible_states is one more option to check
    verbose = False
)
crf.fit(initial_X_train, initial_Y_train)

C:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\sklearn\base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [12]:
labels = list(crf.classes_)
labels.remove('O')
labels

['B-ORG', 'I-ORG', 'B-PER', 'I-PER', 'B-LOC', 'B-MISC', 'I-MISC', 'I-LOC']

In [13]:
# for storing f1 score of testa and testb

predict_f1_testa = []
predict_f1_testb = []

y_preda = crf.predict(X_testa)                                                  ##Iter 0

predict_f1_testa.append(metrics.flat_f1_score(y_testa, y_preda, average='weighted', labels=labels))

print(predict_f1_testa[-1])    

0.5442214555635854


In [14]:
y_predb = crf.predict(X_testb)
predict_f1_testb.append(metrics.flat_f1_score(y_testb, y_predb, average='weighted', labels=labels))      # Return F1 score for sequence items.
                                                              # check sklearn.metrics.f1_score parameters
print(predict_f1_testb[-1])     

0.5523012708545431


In [15]:
# group B and I results
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_testa, y_preda, labels=sorted_labels, digits=3
)) 


C:\ProgramData\Anaconda3\envs\myenv\lib\site-packages\sklearn\utils\validation.py:71: FutureWarning: Pass labels=['B-LOC', 'I-LOC', 'B-MISC', 'I-MISC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


              precision    recall  f1-score   support

       B-LOC      0.453     0.664     0.539       984
       I-LOC      0.384     0.318     0.347       337
      B-MISC      0.234     0.083     0.123       445
      I-MISC      0.280     0.092     0.138       654
       B-ORG      0.709     0.645     0.675      1700
       I-ORG      0.642     0.610     0.626      1366
       B-PER      0.639     0.520     0.574      1222
       I-PER      0.683     0.771     0.724       859

   micro avg      0.592     0.540     0.565      7567
   macro avg      0.503     0.463     0.468      7567
weighted avg      0.570     0.540     0.544      7567



In [16]:
# group B and I results
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_testb, y_predb, labels=sorted_labels, digits=3
))


              precision    recall  f1-score   support

       B-LOC      0.553     0.613     0.581      1084
       I-LOC      0.277     0.182     0.219       325
      B-MISC      0.236     0.074     0.112       339
      I-MISC      0.370     0.102     0.160       557
       B-ORG      0.629     0.643     0.636      1400
       I-ORG      0.616     0.624     0.620      1104
       B-PER      0.680     0.698     0.689       735
       I-PER      0.750     0.841     0.793       634

   micro avg      0.605     0.557     0.580      6178
   macro avg      0.514     0.472     0.476      6178
weighted avg      0.568     0.557     0.552      6178



In [17]:
iteration_num = 0

while predict_f1_testa[-1] < 0.7630 or predict_f1_testb[-1] < 0.7982:
    iteration_num +=1
    print("STARTING iteration {}".format(iteration_num), "\n\n")
    
    X_pool_prob = crf.predict_marginals(X_pool)   

    print("Task 1: storing the sentence according to max uncertainty found within words of sentence......\n\n")

    import operator

    wanted_sent_index = []
    wanted_sent_uncertainty = []
    sent_count = -1

    for sent in X_pool_prob:

        sent_count += 1
        word_count = -1
        meaning_ful_sent = False

        temp_word_uncertainty = []

        for word in sent:

                word_count += 1
                max_dict_key = max(word.items(), key=operator.itemgetter(1))[0]                    # stores class having highest probability

                max_dict_prob = word[max_dict_key]                                                     # stores max prob of abv class
                classification_uncertainty = 1 - max_dict_prob

                temp_word_uncertainty.append(classification_uncertainty)

          
        if len(temp_word_uncertainty) < 2:
            average_of_top_2_LC = 0
        else:    
            first_max_number = max(temp_word_uncertainty)
            temp_word_uncertainty.remove(first_max_number)
            second_max_number = max(temp_word_uncertainty)
            average_of_top_2_LC = (first_max_number + second_max_number)/2                           # average of LC of two words in a sentence
            
        wanted_sent_uncertainty.append(average_of_top_2_LC)
        wanted_sent_index.append(sent_count)

    
    print("Task 1 completed.....\n\n")

    print("Task 2 Finding top 10 most uncertain sentences..... \n")
    top_ten_uncertain_index = sorted(range(len(wanted_sent_uncertainty)), key=lambda i: wanted_sent_uncertainty[i])[-10:]

    print("Task 2 completed.....\n\n")

    print("Task 3 : Remove 10 most uncertain sent from X_pool and add them for training....\n")

    n_labeled_examples = len(X_pool)


    training_indices = top_ten_uncertain_index

    initial_X_train.extend([X_pool[x] for x in training_indices])       #[main_list[x] for x in indexes]
    initial_Y_train.extend([y_pool[x] for x in training_indices])

    # Isolate the non-training examples we'll be querying.
    X_pool = np.delete(X_pool, training_indices).tolist()
    y_pool = np.delete(y_pool, training_indices).tolist()

    print("\t # of sent in X_pool: ", len(X_pool), "\n")
    print("\t # of sent in y_pool: ", len(y_pool), "\n\n")

    print("\t # of sent in X_train: ", len(initial_X_train), "\n")
    print("\t # of sent in y_train: ", len(initial_Y_train), "\n\n")

    print("Task 3 completed.....\n\n")

    print("Task 4: training on new train dataset....\n ")

    crf.fit(initial_X_train, initial_Y_train)
    print("Task 4 completed....\n\n ")

    "Task 5: print and store result.....\n"
    if (predict_f1_testa[-1] < 0.7630):
        y_preda = crf.predict(X_testa)                                   ## Iter 2
        predict_f1_testa.append(metrics.flat_f1_score(y_testa, y_preda, average='weighted', labels=labels))      # Return F1 score for sequence items.
        print("f1 Score for testa is : ", predict_f1_testa[-1], "\n")
        print("\nlen of prdict_f1_testa ", len(predict_f1_testa),"\n")
    else: 
        print("prediction of testa completed....\n")
                                                                                     # check sklearn.metrics.f1_score parameters
    if (predict_f1_testb[-1] < 0.7982):         
        y_predb = crf.predict(X_testb)
        predict_f1_testb.append(metrics.flat_f1_score(y_testb, y_predb, average='weighted', labels=labels))      # Return F1 score for sequence items.
        print("f1 Score for testb is : ", predict_f1_testb[-1], "\n")
        print("\nlen of prdict_f1_testb ", len(predict_f1_testb),"\n")
    else: 
        print("prediction of testa completed....\n")

    print("Task 5 completed of iteration {}....\n\n\n========================================================\n\n\n".format(iteration_num))

    
print("Experiment finished after total {} iterations ".format(iteration_num) )
print("no of sent in training set : {}".format(len(initial_X_train)))

print("Final classification report of testa and testb: \n\n")
print("=================================================")
# group B and I results
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_testa, y_preda, labels=sorted_labels, digits=3
))

# group B and I results
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_testb, y_predb, labels=sorted_labels, digits=3
))

STARTING iteration 1 


Task 1: storing the sentence according to max uncertainty found within words of sentence......


Task 1 completed.....


Task 2 Finding top 10 most uncertain sentences..... 

Task 2 completed.....


Task 3 : Remove 10 most uncertain sent from X_pool and add them for training....

	 # of sent in X_pool:  8230 

	 # of sent in y_pool:  8230 


	 # of sent in X_train:  93 

	 # of sent in y_train:  93 


Task 3 completed.....


Task 4: training on new train dataset....
 
Task 4 completed....

 
f1 Score for testa is :  0.5616581979519867 


len of prdict_f1_testa  2 

f1 Score for testb is :  0.5780292079006509 


len of prdict_f1_testb  2 

Task 5 completed of iteration 1....





STARTING iteration 2 


Task 1: storing the sentence according to max uncertainty found within words of sentence......


Task 1 completed.....


Task 2 Finding top 10 most uncertain sentences..... 

Task 2 completed.....


Task 3 : Remove 10 most uncertain sent from X_pool and add them f

Task 4 completed....

 
f1 Score for testa is :  0.6111681635185505 


len of prdict_f1_testa  12 

f1 Score for testb is :  0.6308354882382393 


len of prdict_f1_testb  12 

Task 5 completed of iteration 11....





STARTING iteration 12 


Task 1: storing the sentence according to max uncertainty found within words of sentence......


Task 1 completed.....


Task 2 Finding top 10 most uncertain sentences..... 

Task 2 completed.....


Task 3 : Remove 10 most uncertain sent from X_pool and add them for training....

	 # of sent in X_pool:  8120 

	 # of sent in y_pool:  8120 


	 # of sent in X_train:  203 

	 # of sent in y_train:  203 


Task 3 completed.....


Task 4: training on new train dataset....
 
Task 4 completed....

 
f1 Score for testa is :  0.6228514247977646 


len of prdict_f1_testa  13 

f1 Score for testb is :  0.631158645783008 


len of prdict_f1_testb  13 

Task 5 completed of iteration 12....





STARTING iteration 13 


Task 1: storing the sentence according t

Task 4 completed....

 
f1 Score for testa is :  0.6398969847807546 


len of prdict_f1_testa  23 

f1 Score for testb is :  0.6705075849100546 


len of prdict_f1_testb  23 

Task 5 completed of iteration 22....





STARTING iteration 23 


Task 1: storing the sentence according to max uncertainty found within words of sentence......


Task 1 completed.....


Task 2 Finding top 10 most uncertain sentences..... 

Task 2 completed.....


Task 3 : Remove 10 most uncertain sent from X_pool and add them for training....

	 # of sent in X_pool:  8010 

	 # of sent in y_pool:  8010 


	 # of sent in X_train:  313 

	 # of sent in y_train:  313 


Task 3 completed.....


Task 4: training on new train dataset....
 
Task 4 completed....

 
f1 Score for testa is :  0.6479029570905778 


len of prdict_f1_testa  24 

f1 Score for testb is :  0.6690851922383579 


len of prdict_f1_testb  24 

Task 5 completed of iteration 23....





STARTING iteration 24 


Task 1: storing the sentence according 

Task 4 completed....

 
f1 Score for testa is :  0.6593706539588193 


len of prdict_f1_testa  34 

f1 Score for testb is :  0.695681966233177 


len of prdict_f1_testb  34 

Task 5 completed of iteration 33....





STARTING iteration 34 


Task 1: storing the sentence according to max uncertainty found within words of sentence......


Task 1 completed.....


Task 2 Finding top 10 most uncertain sentences..... 

Task 2 completed.....


Task 3 : Remove 10 most uncertain sent from X_pool and add them for training....

	 # of sent in X_pool:  7900 

	 # of sent in y_pool:  7900 


	 # of sent in X_train:  423 

	 # of sent in y_train:  423 


Task 3 completed.....


Task 4: training on new train dataset....
 
Task 4 completed....

 
f1 Score for testa is :  0.6627261444984589 


len of prdict_f1_testa  35 

f1 Score for testb is :  0.7060166729465168 


len of prdict_f1_testb  35 

Task 5 completed of iteration 34....





STARTING iteration 35 


Task 1: storing the sentence according t

Task 4 completed....

 
f1 Score for testa is :  0.6835304086222862 


len of prdict_f1_testa  45 

f1 Score for testb is :  0.7268802243643693 


len of prdict_f1_testb  45 

Task 5 completed of iteration 44....





STARTING iteration 45 


Task 1: storing the sentence according to max uncertainty found within words of sentence......


Task 1 completed.....


Task 2 Finding top 10 most uncertain sentences..... 

Task 2 completed.....


Task 3 : Remove 10 most uncertain sent from X_pool and add them for training....

	 # of sent in X_pool:  7790 

	 # of sent in y_pool:  7790 


	 # of sent in X_train:  533 

	 # of sent in y_train:  533 


Task 3 completed.....


Task 4: training on new train dataset....
 
Task 4 completed....

 
f1 Score for testa is :  0.6840051993914353 


len of prdict_f1_testa  46 

f1 Score for testb is :  0.7245968187358873 


len of prdict_f1_testb  46 

Task 5 completed of iteration 45....





STARTING iteration 46 


Task 1: storing the sentence according 

...

Task 1 completed.....


Task 2 Finding top 10 most uncertain sentences..... 

Task 2 completed.....


Task 3 : Remove 10 most uncertain sent from X_pool and add them for training....

	 # of sent in X_pool:  6820 

	 # of sent in y_pool:  6820 


	 # of sent in X_train:  1503 

	 # of sent in y_train:  1503 


Task 3 completed.....


Task 4: training on new train dataset....
 
Task 4 completed....

 
f1 Score for testa is :  0.7535940183681846 


len of prdict_f1_testa  143 

f1 Score for testb is :  0.787878794264516 


len of prdict_f1_testb  143 

Task 5 completed of iteration 142....





STARTING iteration 143 


Task 1: storing the sentence according to max uncertainty found within words of sentence......


Task 1 completed.....


Task 2 Finding top 10 most uncertain sentences..... 

Task 2 completed.....


Task 3 : Remove 10 most uncertain sent from X_pool and add them for training....

	 # of sent in X_pool:  6810 

	 # of sent in y_pool:  6810 


	 # of sent in X_train:  1513 

f1 Score for testa is :  0.7435308475655454 


len of prdict_f1_testa  153 

f1 Score for testb is :  0.7898546893365097 


len of prdict_f1_testb  153 

Task 5 completed of iteration 152....





STARTING iteration 153 


Task 1: storing the sentence according to max uncertainty found within words of sentence......


Task 1 completed.....


Task 2 Finding top 10 most uncertain sentences..... 

Task 2 completed.....


Task 3 : Remove 10 most uncertain sent from X_pool and add them for training....

	 # of sent in X_pool:  6710 

	 # of sent in y_pool:  6710 


	 # of sent in X_train:  1613 

	 # of sent in y_train:  1613 


Task 3 completed.....


Task 4: training on new train dataset....
 
Task 4 completed....

 
f1 Score for testa is :  0.7482213357730969 


len of prdict_f1_testa  154 

f1 Score for testb is :  0.7895541897972568 


len of prdict_f1_testb  154 

Task 5 completed of iteration 153....





STARTING iteration 154 


Task 1: storing the sentence according to max uncerta

Task 4 completed....

 
f1 Score for testa is :  0.7587080397030213 


len of prdict_f1_testa  164 

prediction of testa completed....

Task 5 completed of iteration 163....





STARTING iteration 164 


Task 1: storing the sentence according to max uncertainty found within words of sentence......


Task 1 completed.....


Task 2 Finding top 10 most uncertain sentences..... 

Task 2 completed.....


Task 3 : Remove 10 most uncertain sent from X_pool and add them for training....

	 # of sent in X_pool:  6600 

	 # of sent in y_pool:  6600 


	 # of sent in X_train:  1723 

	 # of sent in y_train:  1723 


Task 3 completed.....


Task 4: training on new train dataset....
 
Task 4 completed....

 
f1 Score for testa is :  0.7591910811195973 


len of prdict_f1_testa  165 

prediction of testa completed....

Task 5 completed of iteration 164....





STARTING iteration 165 


Task 1: storing the sentence according to max uncertainty found within words of sentence......


Task 1 completed

Task 1: storing the sentence according to max uncertainty found within words of sentence......


Task 1 completed.....


Task 2 Finding top 10 most uncertain sentences..... 

Task 2 completed.....


Task 3 : Remove 10 most uncertain sent from X_pool and add them for training....

	 # of sent in X_pool:  6490 

	 # of sent in y_pool:  6490 


	 # of sent in X_train:  1833 

	 # of sent in y_train:  1833 


Task 3 completed.....


Task 4: training on new train dataset....
 
Task 4 completed....

 
f1 Score for testa is :  0.7524259032907543 


len of prdict_f1_testa  176 

prediction of testa completed....

Task 5 completed of iteration 175....





STARTING iteration 176 


Task 1: storing the sentence according to max uncertainty found within words of sentence......


Task 1 completed.....


Task 2 Finding top 10 most uncertain sentences..... 

Task 2 completed.....


Task 3 : Remove 10 most uncertain sent from X_pool and add them for training....

	 # of sent in X_pool:  6480 

	 # of

Task 1 completed.....


Task 2 Finding top 10 most uncertain sentences..... 

Task 2 completed.....


Task 3 : Remove 10 most uncertain sent from X_pool and add them for training....

	 # of sent in X_pool:  6380 

	 # of sent in y_pool:  6380 


	 # of sent in X_train:  1943 

	 # of sent in y_train:  1943 


Task 3 completed.....


Task 4: training on new train dataset....
 
Task 4 completed....

 
f1 Score for testa is :  0.7574620589788863 


len of prdict_f1_testa  187 

prediction of testa completed....

Task 5 completed of iteration 186....





STARTING iteration 187 


Task 1: storing the sentence according to max uncertainty found within words of sentence......


Task 1 completed.....


Task 2 Finding top 10 most uncertain sentences..... 

Task 2 completed.....


Task 3 : Remove 10 most uncertain sent from X_pool and add them for training....

	 # of sent in X_pool:  6370 

	 # of sent in y_pool:  6370 


	 # of sent in X_train:  1953 

	 # of sent in y_train:  1953 


Task 3

In [18]:
import pickle
with open('0_AL_SpanishConLL2002_top2LC_graph_data_testa_f1_pred.pkl', 'wb') as f:
    pickle.dump(predict_f1_testa, f)
    
with open('0_AL_SpanishConLL2002_top2LC_graph_data_testb_f1_pred.pkl', 'wb') as f:
    pickle.dump(predict_f1_testb, f)    
    
with open('0_AL_SpanishConLL2002_top2LC_graph_data_testa_f1_pred.pkl', 'rb') as f:
    mynewlist_a = pickle.load(f)
    
with open('0_AL_SpanishConLL2002_top2LC_graph_data_testb_f1_pred.pkl', 'rb') as f:
    mynewlist_b = pickle.load(f)    
    
print(len(mynewlist_a))
print(len(predict_f1_testa))
print(len(mynewlist_b))
print(len(predict_f1_testb))

191
191
163
163


In [20]:
import numpy as np
Xa = [83+item*10 for item in range(191)]
print(Xa[-1])

1983


In [21]:
import numpy as np
Xa = [83+item*10 for item in range(163)]
print(Xa[-1])

1703
